This notebook comes from https://github.com/chauvinSimon/tri_stats.

## SETUP

Only needed **once**:
- [Create a Google account](https://accounts.google.com/signin). You can use your existing Google account if you already have one. However, since running this code requires storage and access to the account's Google Drive, it is recommended to create a dedicated account specifically for this project.
- [Create a key for the World Triathlon API](https://apps.api.triathlon.org/register) and write it in the next cell. You can use the just created Google address for the registration.

Needed **each time**:
- Run the cells by clicking "play", on their left side. The first one starts the runtime and therefore may take a bit long.

In [ ]:
YOUR_API_KEY = "WRITE YOUR OWN KEY HERE"
# YOUR_API_KEY = "2649776ef9ece4c391003b521cbfce7a"  # example only!

In [ ]:
from pathlib import Path
from google.colab import drive

In [ ]:
drive_dir = Path("/content/drive")
drive_nb_dir = drive_dir / "MyDrive/Colab Notebooks"
project_dir = drive_nb_dir / "tri_stats"

repo_url = "https://github.com/chauvinSimon/tri_stats.git"

A pop-up window should open:
> _"Permit this notebook to access your Google Drive files?"_

You should:
- Click `"Connect to Google Drive"`.
- Select your Google account.
- Click `Continue` on `Sign in to Google Drive for desktop`.
- Click `Select all` on `Select what Google Drive for desktop can access`. _(you can revoke this grant in your Google setting, by looking for `Google Drive for desktop` in `Data from apps and services you use`)_.
- Scroll down and validate with `Continue`.

In [ ]:
if not drive_dir.exists():
  print("mounting drive")
  drive.mount('/content/drive')

In [ ]:
if not project_dir.exists():
    # Convert path to a string and quote it for bash commands. Against space in "Colab Notebooks"
    quoted_project_dir = f'"{project_dir}"'
    print(f"cloning repo from: {repo_url}")
    !git clone {repo_url} {quoted_project_dir}

In [ ]:
%cd {project_dir}
!git status

In [ ]:
%cd {project_dir}
!git pull origin main

In [ ]:
%cd {project_dir}/scripts
assert Path().resolve() == project_dir / "scripts"
%ls

In [ ]:
api_key_path = project_dir / "api_key.txt"
if (not api_key_path.exists()) or (api_key_path.read_text() != YOUR_API_KEY):
    print(f"Writing key to local file: {YOUR_API_KEY}")
    api_key_path.write_text(YOUR_API_KEY)

## USAGE

At this point, you are ready to:
- Collect data from the World Triathlon API.
- Process and clean it up.
- Format it to a table.
- Export it to a csv file.

In [ ]:
from utils import load_config
from utils_events import get_events_df

In [ ]:
export_dir = project_dir / "ignored" / "exports"
export_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
config = load_config()
events_config = config["events"]

In [ ]:
# just for quick test: querying a narrow range of dates
events_config["query"]["start_date"] = "2024-05-01"
events_config["query"]["end_date"] = "2024-05-31"

The next cell makes requests to the API.
- It takes time the first time.
- It is much faster then, because the results of the requests are saved.

In [ ]:
df = get_events_df(events_config)

The next cell shows the created table (or a section of it).

In [ ]:
df

In [ ]:
list(df.columns)

Example to apply filters:

In [ ]:
df[
    (df["event_category"] == "wcs") &
    df["wetsuit_w"] &
    ~df["wetsuit_m"]
][["event_title", "prog_distance_category", "wetsuit_w", "wetsuit_m", "swim_all_w", "swim_all_m", "swim_diff_percent"]]

## EXPORT TABLE

The next cell saves the table (`df`) to your Drive:
- Go to https://drive.google.com/drive/my-drive.
- You should find the saved .csv under `My Drive / Colab Notebooks / tri_stats / ignored / exported`.
- You may need to refresh the page.
- Before downloading the .csv, you can have a look: `Open with` -> `Google Sheets`.

In [ ]:
df.to_csv(export_dir / "events.csv")

## QUIT

- Press `ctrl+s` to save the changes of the notebook.
- Next time, you can open the notebook directly at https://drive.google.com/drive/my-drive, under `My Drive / Colab Notebooks / Copy of main.ipynb`.